In [57]:
import pandas as pd
import requests
import simplejson as json
from shapely.geometry import shape
import geopandas as gpd
import json
import matplotlib as plt
from matplotlib import pyplot as plt
import mapclassify

from descartes import PolygonPatch
import numpy as np
import urllib.request

from PIL import Image
import glob
import seaborn as sns
from pandas import read_excel


In [58]:
my_sheet = 'Sheet1' # change it to your sheet name, you can find your sheet name at the bottom left of your excel file
file_name = 'oil-mali.xlsx' # change it to the name of your excel file
license = pd.read_excel(file_name, sheet_name = my_sheet)
license.columns.name = license.index.name
license.index.name = None
license = license[(license['Start Date'] > '2012-1-1') & (license['Start Date'] <= '2022-6-10')]

license = license.replace(r'\n','-', regex=True) 
license['Regions'] = license['Regions'].str.split('-').str[-1].str.strip()
license['Regions'] = license['Regions'].str.rsplit(' (', 1).str.get(0)
license['Regions'] = license['Regions'].str.split('Région de').str[-1].str.strip()
license['year'] = pd.DatetimeIndex(license['Start Date']).year
# license['Start Date'] = license['Start Date'].str.split('-').str[-1].str.strip()

license.head()

,Start Date,License Code,Owner,Type,Status,Minerals,Regions,year
0,2021-08-23,SS 2703/21,"SIDIBE, Amadou",Autorisation d'ouverture de station service et...,Active License,Station Service,Bougouni,2021
1,2021-07-28,SS 2618/21,ALLIANCE PETROLEUM,Autorisation d'ouverture de station service et...,Active License,Dépôt d'hydrocarbures,Koulikoro,2021
2,2021-04-06,SS 2617/21,VIVO ENERGY MALI SA,Autorisation d'ouverture de station service et...,Active License,Station Service,Bamako,2021
3,2020-12-14,SS 2612/20,SOMAYAF-SA,Autorisation d'ouverture de station service et...,Active License,Station Service,Bamako,2020
4,2020-12-14,SS 2611/20,SOMAYAF-SA,Autorisation d'ouverture de station service et...,Active License,Station Service,Bamako,2020


In [59]:
new = license.groupby(['Regions','year', "Minerals"]).size().reset_index().rename(columns={0:'count'})
new['longitude'] = 0
def get_long(param):
    if param == "Doumanténé":
        param = "Bagoé"
    payload = {'access_key': '83289c2a9befcae08f3c9f21f3a1b7bd', 'query': param, 'country':"ML"}
    response_API = requests.get('http://api.positionstack.com/v1/forward', params=payload)
    response = response_API.text
    parse_json = json.loads(response)
    long = parse_json["data"][0]["longitude"]
    return parse_json 

for index in new.index:
            new.loc[index, "longitude"] = get_long(new.loc[index, "Regions"])["data"][0]["longitude"]
            new.loc[index, "latitude"] = get_long(new.loc[index, "Regions"])["data"][0]["latitude"]

# for index in new.index:
#     if new.loc[index, 'Regions'] == "Adrar des Iforas":
#         new.loc[index, "longitude"] = get_long("Adrar des Iforas")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("Adrar des Iforas")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "Bagoé":
#         new.loc[index, "longitude"] = get_long("Bagoé")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("Bagoé")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "Doumanténé":
#         new.loc[index, "longitude"] = get_long("Doumanténé")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("Doumanténé")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "Kangaba":
#         new.loc[index, "longitude"] = get_long("Kangaba")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("Kangaba")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "Katiolo":
#         new.loc[index, "longitude"] = get_long("Katiolo")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("Katiolo")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "Kayes":
#         new.loc[index, "longitude"] = get_long("Kayes")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("Kayes")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "test":
#         new.loc[index, "longitude"] = get_long("test")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("test")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "test":
#         new.loc[index, "longitude"] = get_long("test")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("test")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "test":
#         new.loc[index, "longitude"] = get_long("test")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("test")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "test":
#         new.loc[index, "longitude"] = get_long("test")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("test")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "test":
#         new.loc[index, "longitude"] = get_long("test")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("test")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "test":
#         new.loc[index, "longitude"] = get_long("test")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("test")["data"][0]["latitude"]
#     if new.loc[index, 'Regions'] == "test":
#         new.loc[index, "longitude"] = get_long("test")["data"][0]["longitude"]
#         new.loc[index, "latitude"] = get_long("test")["data"][0]["latitude"]
new


,Regions,year,Minerals,count,longitude,latitude
0,Bamako,2012,Station Service,4,-8.000000,12.650000
1,Bamako,2013,Dépôt d'hydrocarbures,1,-8.000000,12.650000
2,Bamako,2013,Station Service,2,-8.000000,12.650000
3,Bamako,2014,Station Service,2,-8.000000,12.650000
4,Bamako,2015,Station Service,1,-8.000000,12.650000
5,Bamako,2016,Station Service,2,-8.000000,12.650000
6,Bamako,2017,Dépôt d'hydrocarbures,2,-8.000000,12.650000
7,Bamako,2017,Station Service,8,-8.000000,12.650000
8,Bamako,2018,Station Service,2,-8.000000,12.650000
9,Bamako,2019,Station Service,11,-8.000000,12.650000


In [60]:
count = new["count"].sum()
count

138

In [61]:
nta = pd.read_json( r'./cercles.json' )
nta['geometry'] = nta['geometry'].apply(shape)
nta = pd.concat([nta.drop(['properties','id'], axis=1), nta['properties'].apply(pd.Series)], axis=1)
mali = gpd.GeoDataFrame(nta).set_geometry('geometry')
mali


,type,geometry,geometry_name,bbox,id_0,iso,name_0,id_1,name_1,id_2,name_2,id_3,name_3,id_4,name_4,varname_4,ccn_4,cca_4,type_4,engtype_4
0,Feature,"MULTIPOLYGON (((-7.90780 12.64515, -7.91502 12...",geom,"[-7.99214697, 12.63429737, -7.90779686, 12.711...",138,MLI,Mali,1,Bamako,1,Bamako,1,Kalban-Coro,1,Commune I,None,0,None,Commune,Commune
1,Feature,"MULTIPOLYGON (((-7.95253 12.63430, -7.95820 12...",geom,"[-7.998981, 12.62709999, -7.95231104, 12.68022...",138,MLI,Mali,1,Bamako,1,Bamako,1,Kalban-Coro,2,Commune II,None,0,None,Commune,Commune
2,Feature,"MULTIPOLYGON (((-7.99214 12.63013, -7.99266 12...",geom,"[-8.05370808, 12.62367916, -7.98262501, 12.680...",138,MLI,Mali,1,Bamako,1,Bamako,1,Kalban-Coro,3,Commune III,None,0,None,Commune,Commune
3,Feature,"MULTIPOLYGON (((-8.00379 12.62368, -8.00356 12...",geom,"[-8.073596, 12.56077576, -8.00356102, 12.66203...",138,MLI,Mali,1,Bamako,1,Bamako,1,Kalban-Coro,4,Commune IV,None,0,None,Commune,Commune
4,Feature,"MULTIPOLYGON (((-8.00893 12.57583, -8.01029 12...",geom,"[-8.03932667, 12.52526569, -7.96626806, 12.630...",138,MLI,Mali,1,Bamako,1,Bamako,1,Kalban-Coro,5,Commune V,None,0,None,Commune,Commune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,Feature,"MULTIPOLYGON (((-2.49073 16.80968, -2.50215 16...",geom,"[-2.68438792, 16.80275345, -0.8680289999999999...",138,MLI,Mali,9,Timbuktu,50,Tombouctou,286,Ber,700,Ber,None,0,None,Commune,Commune
700,Feature,"MULTIPOLYGON (((-2.51428 16.72845, -2.54989 16...",geom,"[-2.92217803, 16.64541054, -2.48004198, 17.090...",138,MLI,Mali,9,Timbuktu,50,Tombouctou,287,Bourem-Inali,701,Bourem-Inaly,None,0,None,Commune,Commune
701,Feature,"MULTIPOLYGON (((-2.04144 19.99941, -2.05587 19...",geom,"[-3.48680592, 17.08261681, -2.04144192, 20.000...",138,MLI,Mali,9,Timbuktu,50,Tombouctou,288,Tinaguelhaj,702,Salam,None,0,None,Commune,Commune
702,Feature,"MULTIPOLYGON (((-3.01341 16.78969, -3.01176 16...",geom,"[-3.02941895, 16.68831062, -2.95813489, 16.789...",138,MLI,Mali,9,Timbuktu,50,Tombouctou,289,Tombouctou,704,Tombouctou,None,0,None,Urban Commune,Urban Commune


In [66]:
left, width = .45, .5
bottom, height = .45, .5
right = left + width
top = bottom + height

left2, width2 = .66, .2
bottom2, height2 = .66, .2
right2 = left2 + width2
top2 = bottom2 + height2

left3, width3 = .10, .70
bottom3, height3 = .10, .70
right3 = left3 + width3
top3 = bottom3 + height3

left4, width4 = .10, .1
bottom4, height4 = .10, .1
right4 = left4 + width4
top4 = bottom4 + height4

years = [2012, 2013, 2014, 2015, 2016,
         2017, 2018, 2019, 2020, 2021, 2022]

for i in years:
    left, width = .45, .5
    bottom, height = .45, .5
    right = left + width
    top = bottom + height

    points = gpd.GeoDataFrame(
    new, geometry=gpd.points_from_xy(new.longitude, new.latitude))

    ax2 = mali.plot(figsize=(10, 10), edgecolor="face", lw=1,
                fc="lightblue")
    ax2.set_title('Pétrole et gaz: Nombre de permis/autorisation par année', y=0, pad=-25, verticalalignment="top", fontsize=15)
    year = i
    name = "petrole" + str(i)
    ax2.text(right, top, year,
             horizontalalignment='right',
             verticalalignment='top',
             transform=ax2.transAxes,
             fontsize=40)

    ax2.text(0.81, .82, str(new.loc[(new['year'] == year), 'count'].sum()),
             horizontalalignment='center',
             verticalalignment='center',
             transform=ax2.transAxes,
             color='blue',
             fontsize=40)

    # ax2.text(0.2, 0.87, "Bauxite: " + str(new.loc[(new['year'] == year) & (new['Minerals'] == 'Bauxite'), 'count'].sum()),
    #          horizontalalignment='right',
    #          verticalalignment='top',
    #          transform=ax2.transAxes,
    #          fontsize=30,
    #          color='red')

    # ax2.text(0.2, 0.78, "Lithium: " + str(new.loc[(new['year'] == year) & (new['Minerals'] == 'Lithium'), 'count'].sum()),
    #          horizontalalignment='right',
    #          verticalalignment='top',
    #          transform=ax2.transAxes,
    #          fontsize=30,
    #          color='blue')

    points[(points.year == year)].plot(
        ax=ax2, marker='o', color='blue', markersize=new["count"]*80).set_axis_off()
    points[(points.year == year) & (points.Minerals == "Bauxite")].plot(
        ax=ax2, marker='o', color='red', markersize=new["count"]*40).set_axis_off()
    points[(points.year == year) & (points.Minerals == "Lithium")].plot(
        ax=ax2, marker='o', color='blue', markersize=new["count"]*40).set_axis_off()


    img = Image.open("1960.jpg")
    plt.figimage(img, xo=400, yo=150)

    plt.savefig('./python/' + name + '.png',  facecolor="white")
    plt.clf()
    


/usr/local/lib/python3.9/site-packages/geopandas/plotting.py:681: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/geopandas/plotting.py:681: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/geopandas/plotting.py:681: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/geopandas/plotting.py:681: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/geopandas/plotting.py:681: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/geopandas/plotting.py:681: UserWarning: The GeoDataFrame you are attemp

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

In [68]:
fp_in = "./python/petrole*.png"
fp_out = "./python/petrole-2012-2022.gif"

imgs = (Image.open(f) for f in sorted(glob.glob(fp_in)))
img = next(imgs)  # extract first image from iterator
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=2000, loop=1)